# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.libor import *
from financepy.finutils import *
from financepy.products.libor.FinLiborCurve import FinLiborCurve
from financepy.market.curves.FinInterpolate import FinInterpTypes

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Building a Libor Discount Curve

In [3]:
valuationDate = FinDate(28, 2, 2014)
settlementDate = FinDate(4, 3, 2014)

We have a vector of dates and zero rates

In [5]:
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinLiborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = FinLiborSwapTypes.PAYER
fixedDCCType = FinDayCountTypes.ACT_365F
fixedFreqType = FinFrequencyTypes.SEMI_ANNUAL

swap = FinLiborSwap(settlementDate, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

In [9]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")
swapFixedCoupon = 0.040852
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FinFrequencyTypes.QUARTERLY
swapFloatDayCountType = FinDayCountTypes.ACT_360
swapNotional = ONE_MILLION
swapType = FinLiborSwapTypes.PAYER  
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.NONE
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

In [10]:
swaption = FinLiborSwaption(settlementDate,
                            exerciseDate,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

## Valuation using Black's Model

In [11]:
model = FinModelBlack(0.1533)

In [12]:
swaption.value(settlementDate, liborCurve, model)

23165.212640758255

## Valuation using Shifted Black

In [9]:
model = FinModelBlackShifted(0.1533, -0.008)

In [10]:
swaption.value(settlementDate, liborCurve, model)

27688.140836760464

##  Valuation using SABR

In [11]:
model = FinModelSABR(0.132, 0.5, 0.5, 0.5)

In [12]:
swaption.value(settlementDate, liborCurve, model)

104548.07605710132

## Valuation using Shifted SABR

In [13]:
model = FinModelSABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [14]:
swaption.value(settlementDate, liborCurve, model)

163429.25215071545

## Valuation using Hull-White

In [15]:
model = FinModelRatesHW(0.01, 0.02)

In [16]:
swaption.value(settlementDate, liborCurve, model)

34197.6767271399

## Valuation using Black-Karasinski

In [17]:
model = FinModelRatesBK(0.2, 0.05)

In [18]:
swaption.value(settlementDate, liborCurve, model)

** USE WITH CAUTION AS TREE MAY NEED TWEAKS FOR ACCRUED **


23499.014155799534

## Internals

In [19]:
print(swaption)

OBJECT TYPE: FinLiborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAPTION TYPE: FinLiborSwaptionTypes.PAYER
MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PV01: 4.161673891171295
FWD SWAP RATE: 4.086151574330605
FWD DF TO EXPIRY: 0.9237933367937652


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [20]:
swaption.printSwapFixedLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
SAT 04 SEP 2021  0.5000000     20426.00   0.84102606     17178.80     89353.55
FRI 04 MAR 2022  0.5000000     20426.00   0.82477877     16846.93    106200.48
SUN 04 SEP 2022  0.5000000     20426.00   0.80771756     16498.44    122698.92
SAT 04 MAR 2023  0.5000000     20426.00   0.79039583     16144.63    1388

Copyright (c) 2020 Dominic O'Kane